In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset   # 데이터를 읽어옴
import torchvision.transforms as transforms    # 불러온 이미지를 필요에 따라 변환
from torch.utils.data import DataLoader   # 효율적인 학습을 위해 데이터를 어떤 규칙에 따라 정렬 또는 섞음

# hyperparameter 설정
batch_size = 256
learning_rate = 0.0002
num_epoch = 10


In [9]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test   = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)
# train = True(학습데이터), False(테스트 데이터)
# transform 데이터변형 (이미지 변형), transform.ToTensor():파이토치텐서로 변형
# target_transform 라벨 변형
# download : 현재 경로에 mnist데이터가 없을 경우 다운로드

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader  = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)
# dset.MNIST를 통해 정리된 데이터를 batch_size 개수만큼 묶는다
# shuffle : 셔플 여부
# num_workers : 데이터를 묶을때 사용할 프로세스 개수
# drop_last : 묶고 남는 데이터를 버릴지 여부
# train_loader test_loader는 순차적으로 모델에 데이터 전달


In [10]:
# 합성곱 신경망(CNN) 모델
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # super : CNN클래스의 부모 클래스인 nn.Module 초기화
        
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),    # 1,28,28 => 16,24,24
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),            # 16,24,24 => 32,20,20
            nn.MaxPool2d(2,2),    # 32,20,20 => 32,10,10
            nn.Conv2d(32,64,5),
            nn.ReLU(),            # 32,10,10 => 64,6,6
            nn.MaxPool2d(2,2)     # 64,6,6 => 64,3,3
        )
        # Conv2d(in_channels, out_channels, kernel_size)
        # stride=1, padding=0 기본값
        # MaxPool2d(kernel_size, stride)
        # kernel_size : 한번에 훑는 영역의 크기
        
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100),
            nn.ReLU(),
            nn.Linear(100,10)
        )
        # [batch_size, 10]
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out


In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [18]:
loss_arr =[]
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
        if j%1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3067, grad_fn=<NllLossBackward>)
tensor(0.1900, grad_fn=<NllLossBackward>)
tensor(0.1070, grad_fn=<NllLossBackward>)
tensor(0.0736, grad_fn=<NllLossBackward>)
tensor(0.0724, grad_fn=<NllLossBackward>)
tensor(0.0559, grad_fn=<NllLossBackward>)
tensor(0.0462, grad_fn=<NllLossBackward>)
tensor(0.0262, grad_fn=<NllLossBackward>)
tensor(0.0452, grad_fn=<NllLossBackward>)
tensor(0.0352, grad_fn=<NllLossBackward>)


In [19]:
# 테스트 데이터로 정확도 측정
correct = 0
total = 0

with torch.no_grad():   # 인퍼런스 모드를 위해 no_grad(기울기 계산안함)
    for image, label in test_loader:   # 테스트로더에서 이미지와 정답 불러오기
        x = image.to(device)
        y_ = label.to(device)
        
        output = model.forward(x)     # 모델을 데이터에 넣고 결과 얻음
        _,output_index = torch.max(output, 1)    # 최댓값 및 최댓값 인텍스를 뽑아냄
        total += label.size(0)
        correct += (output_index == y_).sum().float()
        # 모델의 결과가 최댓값 인덱스와 라벨이 일치하는 개수를 correct에 추가
        
    print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 98.89823913574219
